We extract the data from the web using an API

In [185]:
import pandas as pd
from sodapy import Socrata

# Replace None with your app token "AVDM_Competencies" for authenticated access
client = Socrata("analisi.transparenciacatalunya.cat", "5fdymyZntjECVai1ZmolqKBeG")

# Fetch the first 2000 results as JSON and convert them to a Python list of dictionaries
results = client.get("rk5x-gny6",limit=100000) #"rk5x-gny6" esto es de la base de datos

# Convert to pandas DataFrame
comp = pd.DataFrame.from_records(results)

In [208]:
#Check columns
comp.columns

Index(['codi_alumne', 'pcat', 'pcat_cl', 'pcat_ee', 'pcast', 'pcast_cl',
       'pcast_ee', 'pmat', 'pmat_nc', 'pmat_efm', 'pmat_cr', 'pang', 'pang_co',
       'pang_ce', 'pang_ee', 'pcat_co', 'pcast_co', 'pmed'],
      dtype='object')

In [189]:
columns =['codi_alumne','pcat', 'pcat_cl', 'pcat_ee', 'pcast', 'pcast_cl',
       'pcast_ee', 'pmat', 'pmat_nc', 'pmat_efm', 'pmat_cr', 'pang', 'pang_co',
       'pang_ce', 'pang_ee', 'pcat_co', 'pcast_co', 'pmed']

In [210]:
#Select columns I want and dropna
comp = comp[columns]
comp = comp.dropna()

In [214]:
# Checked repeated codes
repeated = comp['codi_alumne'].value_counts()
no_unique = repeated[repeated > 1].index
unique = repeated[repeated == 1]
# The maximum repetition is two

El que vull primer és trobar una separació d'assignatures segons les intell·ligències múltiples, tenir una agrupació i llavors hauré de fer la mitjana per alumne de la nota en les diferents assignatures. Llavors hauré de trobar una manera de mirar les diferències.
* Lingüística: PCAT, PCAST, PANG
* Lògico-matemàtica: PMAT_NC, PMAT_CR
* Naturalista: PMED
* Espaial: PMAT_EFM


In [195]:
#Make sure columns are numerical
col_num = ['pcat', 'pcast', 'pang', 'pmat_nc', 'pmat_efm', 'pmat_cr', 'pmed']
comp[col_num] = comp[col_num].apply(pd.to_numeric, errors='coerce')

# Drop nan
comp = comp.dropna(subset=col_num)

# Made the mean of marks for repeated student codes 
df_mean = comp.groupby('codi_alumne').agg({
    'pcat': 'mean',  
    'pcast': 'mean',
    'pang': 'mean',
    'pmat_nc': 'mean',
    'pmat_efm': 'mean',
    'pmat_cr': 'mean',
    'pmed': 'mean',
}).reset_index()

# Mostrar el resultat
print(df_mean)


       codi_alumne  pcat  pcast    pang  pmat_nc  pmat_efm  pmat_cr   pmed
0      10025180886  49.5   56.0   57.08    71.43     90.91    55.56  44.03
1      10025641164  77.8   77.9  100.00    85.71     54.55    55.56  72.92
2      10025652386  79.5   84.8   88.67    92.86     63.64    55.56  69.86
3      10025664607  57.2   55.0   57.25    64.29     54.55    55.56  53.33
4      10025675828  55.5   52.7   60.17   100.00    100.00    77.78  64.03
...            ...   ...    ...     ...      ...       ...      ...    ...
92661    999814386  85.7   90.6   90.75   100.00    100.00   100.00  94.44
92662    999826608  49.0   48.0   31.25    71.43     27.27    66.67  66.25
92663    999837829  79.8   76.2   72.50    80.00     70.00    66.67  74.44
92664    999848162  83.5   76.6   47.50    80.00     90.00    55.56  78.61
92665    999859383  69.0   61.9   59.60    86.67     60.00    33.33  58.06

[92666 rows x 8 columns]


In [216]:
# Fer el promig de columnes segons la meua classificació de intel·ligències múltiples, separaré segons la classificació